In [1]:
from selenium import webdriver
import time
from bs4 import BeautifulSoup
import re
import pandas as pd
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.chrome.options import Options

In [2]:
url='https://you.ctrip.com/sight/qingdao5/1266.html#ctm_ref=www_hp_bs_lst'

In [3]:
browser = webdriver.Chrome()
browser.get(url)

In [4]:
name=browser.find_element_by_xpath('//*[@id="__next"]/div[3]/div/div[3]/div[2]/div[1]/h1').get_attribute('innerHTML')
score=float(browser.find_element_by_xpath('//*[@id="__next"]/div[3]/div/div[3]/div[2]/div[2]/div/p[1]').get_attribute('innerHTML'))

In [5]:
comments_total=0
comments=[]
date=[]
good=[]
stars=[]
yeshu=1

In [6]:
while True:
    print(yeshu)
    table = browser.find_element_by_class_name(
        'commentList').get_attribute('innerHTML')
    soup = BeautifulSoup(table, "html.parser")
    parts = soup.find_all(attrs={'class': 'commentItem'})
    for part in parts:
        date_ = re.search(r'<div class="commentTime">(.*?)</div>',
                          str(part), re.M | re.I | re.DOTALL).group(1)
        if '</span>点赞</span>' not in str(part):
            good_ = re.search(r'<span class="common-iconfont toolsIcon votedIcon">.*?</span>(.*?)</span>',
                              str(part), re.M | re.I | re.DOTALL).group(1)
        else:
            good_ = '0'
        comments_ = re.search(
            r'<div class="commentDetail">(.*?)</div>', str(part), re.M | re.I | re.DOTALL).group(1)
        try:
            stars_ = float(re.search(r'.png"/>(.*?)<!-- -->分 <!-- -->',
                                     str(part), re.M | re.I | re.DOTALL).group(1))
        except:
            stars_ = score
        good.append(good_)
        stars.append(stars_)
        date.append(date_.replace('-', r'/'))
        comments.append(comments_)
        comments_total += 1
    btn = browser.find_element_by_class_name('ant-pagination-next')
    if btn.get_attribute('aria-disabled') == 'false':
        time.sleep(15)
        btn.click()
        time.sleep(20)
        yeshu += 1
    else:
        print('点击结束')
        break

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
点击结束


In [7]:
data={'评论':comments,'得分':stars,'时间':date,'点赞':good}

In [8]:
df=pd.DataFrame(data)

In [9]:
df

,评论,得分,时间,点赞
0,小青岛公园位于鲁迅公园附近，可以近距离接触大海，岛上有很多红色礁石，另外还有德国建筑群，我们...,5.0,2021/07/18,0
1,相信有许多游客都去过青岛旅游，不知道大家有没有听说过这个景点，如果有兴趣去青岛旅游的话，一定...,4.0,2021/05/17,6
2,从栈桥过来走路10多分钟，走过去的时候到大门才发现5点就关门了，不过从滨河路到大门这一届路很...,5.0,2021/04/21,2
3,门票十元，是最值得去的地方。青岛之所以叫青岛，就是源于这个小青岛。登上岛，可以看见四周海景，...,5.0,2021/07/17,0
4,里面很大，能逛上一下午，空气好海水蓝，在里面拍照片原片直出！门票只要十块钱，学生票半价奥。注...,5.0,2021/03/05,0
...,...,...,...,...
681,灯塔没有开放。绕着小岛走了一圈，看看退役的舰船，看看礁石海水，没有多大的惊喜。是个可去可不去...,4.4,2009/07/22,0
682,好小的一个岛，真没啥好玩的，除了在边上抓抓小蟹。,4.4,2009/10/08,2
683,就是一个普通的海边公园，真的没什么意思，有时可以看到军舰。,4.4,2016/12/20,1
684,远眺...门票15元 貌似去那里的人不是很多,4.4,2009/08/28,1


In [10]:
df.to_csv('评论数据/'+name+'.csv',encoding='utf_8_sig')